In [1]:
from deepcave import Recorder, Objective
from deepcave.runs.converters.deepcave import DeepCAVERun
from deepcave.plugins.summary.footprint import FootPrint
import pandas as pd
from arlbench.core.algorithms import DQN, PPO, SAC
from pathlib import Path
from ConfigSpace import ConfigurationSpace, Float, Categorical
import math
import os

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
replacement_dict = {"atari_qbert": "Atari", 'atari_double_dunk': "Atari", 'atari_phoenix': "Atari", 'atari_this_game': "Atari", 'atari_battle_zone': "Atari", 
                    'box2d_lunar_lander': "Box2d", "box2d_bipedal_walker": "Box2d", 'cc_acrobot': "CC", 'cc_cartpole': "CC", 'cc_mountain_car': "CC", "cc_pendulum": "CC", 'cc_continuous_mountain_car': "CC",
                    'minigrid_door_key': "XLand", 'minigrid_empty_random': "XLand", 'minigrid_four_rooms': "XLand", 
                    'minigrid_unlock': "XLand", "brax_ant": "Brax", "brax_halfcheetah": "Brax", "brax_hopper": "Brax", "brax_humanoid": "Brax"}
algorithms = {"dqn": DQN, "ppo": PPO, "sac": SAC}
envs = ['box2d_bipedal_walker']
        #'box2d_lunar_lander', 'box2d_bipedal_walker', 'cc_acrobot', 'cc_cartpole', 'cc_mountain_car', 'cc_pendulum', 'cc_continuous_mountain_car',
        #'minigrid_door_key', 'minigrid_empty_random', 'minigrid_four_rooms', 'minigrid_unlock', 'brax_ant', 'brax_halfcheetah', 'brax_hopper', 'brax_humanoid']


In [30]:
def data_to_deepcave(algorithm, domain=False, save_path=None):
    target_path = f"../results_combined/sobol/{domain}_{algorithm}.csv"
    try:
        all_configs = pd.read_csv(target_path)
    except:
        print(f"Could not read {target_path}")
        return

    default_configspace = algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names()
    configspace = ConfigurationSpace()
    for c in default_configspace:
        if c in ["buffer_prio_sampling", "use_target_network", "alpha_auto", "normalize_advantage"]:
            hp = Categorical(c, [True, False])
            configspace.add_hyperparameter(hp)
        else:
            key = [a for a in all_configs.keys() if c in a]
            if len(key) > 0:
                key = key[0]
                hp_min = all_configs[key].min()
                hp_max = all_configs[key].max()
                bounds = (min(0, hp_min-0.1*hp_min), hp_max+0.1*hp_max)
                hp = Float(c, bounds=bounds)
                configspace.add_hyperparameter(hp)
    default = configspace.get_default_configuration()
    accuracy = Objective("reward_mean", lower=min(all_configs["performance"]), upper=max(all_configs["performance"]), optimize="upper")
    save_path = save_path if save_path else f"deepcave_logs/{algorithm}_{domain}"

    with Recorder(configspace, objectives=[accuracy], save_path=save_path) as r:
        for index, d in all_configs.iterrows():
            current_hps = d
            config = default
            for c in current_hps.keys():
                key = c.split(".")[-1]
                print(key)
                if key in config.keys():
                    if math.isnan(current_hps[c]):
                        current_hps[c] = 0
                    try:
                        value = float(current_hps[c])
                    except:
                        current_hps[c]
                    config[key] = value
            r.start(config=config, budget=1, seed=d["seed"])
            r.end(costs=current_hps["performance"], budget=1, seed=d["seed"])
    return save_path

In [31]:
def get_footprint(path):
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))
    objective_id = run.get_objective_ids()[0]
    budget_id = run.get_budget_ids()[-1]

    # Instantiate the plugin
    plugin = FootPrint()

    inputs = plugin.generate_inputs(
        objective_id=objective_id,
        budget_id=budget_id,
        details=True,
        show_supports=True,
        show_borders=True,
    )
    # Note: Filter variables are not considered.
    outputs = plugin.generate_outputs(run, inputs)

    # Finally, you can load the figure. Here, the filter variables play a role.
    # Alternatively: Use the matplotlib output (`load_mpl_outputs`) if available.
    figure = plugin.load_outputs(run, inputs, outputs)  # plotly.go figure
    return figure

In [32]:
algorithm = "ppo"
deepcave_paths = []
for env in envs:
   if not os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
       print(f"{env} not found")

for env in envs:
   if not os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
       data_path = data_to_deepcave(algorithm, domain=env)
       deepcave_paths.append(data_path)

box2d_bipedal_walker not found
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_advantage
vf_clip_eps
vf_coef
seed
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_advantage
vf_clip_eps
vf_coef
seed
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_advantage
vf_clip_eps
vf_coef
seed
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_advantage
vf_clip_eps
vf_coef
seed
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_advantage
vf_clip_eps
vf_coef
seed
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_advantage
vf_clip_eps
vf_coef
seed
run_id
budget
performance
clip_eps
ent_coef
gae_lambda
learning_rate
max_grad_norm
minibatch_size
normalize_adv

In [13]:
paths = []
for env in envs:
   if os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
       paths.append(f"deepcave_logs/{algorithm}_{env}")
    
for data_path in paths:
    print(data_path)
    try:
        img = get_footprint(Path(data_path) / "run_1")
        img[0].write_image(data_path + "/footprint.png")
    except Exception as e:
        print("Error")
        print(e)

deepcave_logs/ppo_box2d_bipedal_walker


100%|█████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 1742.07it/s]

deepcave.evaluators.footprint (INFO): Added 256 configurations.
deepcave.evaluators.footprint (INFO): Starting to calculate distances and add border and random configurations...


deepcave.evaluators.footprint (INFO): Found 600 configurations...
deepcave.evaluators.footprint (INFO): Found 800 configurations...
deepcave.evaluators.footprint (INFO): Found 900 configurations...
deepcave.evaluators.footprint (INFO): Found 1000 configurations...
deepcave.evaluators.footprint (INFO): Found 1200 configurations...
deepcave.evaluators.footprint (INFO): Found 1300 configurations...
deepcave.evaluators.footprint (INFO): Found 1400 configurations...
deepcave.evaluators.footprint (INFO): Found 1500 configurations...
deepcave.evaluators.footprint (INFO): Found 1600 configurations...
deepcave.evaluators.footprint (INFO): Found 1700 configurations...
deepcave.evaluators.footprint (INFO): Found 1800 configurations...
deepcave.evaluators.footprint (INFO): Found 1900 configurations...
deepcave.evaluators.footprint (INFO): Found 2000 configurations...
deepcave.evaluators.footprint (INFO): Found 2100 configurations...
deepcave.evaluators.footprint (INFO): Found 2200 configurations..

In [38]:
for data_path in paths:
    if not os.path.isfile(data_path + "/footprint.png"):
        print(f"Footprint for {data_path} not found")